<a href="https://colab.research.google.com/github/sundaybest3/NounSmart/blob/main/nounsmart_irregular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio pandas

In [ ]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file from the provided URL
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart/refs/heads/main/Irregular_Nouns.csv"
df = pd.read_csv(csv_url)

# Count the number of items in each Level
level_counts = df["Level"].value_counts()
levels_with_counts = [
    f"{level} ({level_counts.get(level, 0)} items)"
    for level in ["easy_peasy", "easy", "normal", "challenging", "super_challenging"]
]

# Function to initialize user-specific state
def initialize_user_state():
    return {
        "nickname": None,
        "remaining_nouns": pd.DataFrame(),
        "current_level": None,
        "score": 0,
        "trials": 0,
        "current_index": -1,
        "level_scores": {level: {"score": 0, "trials": 0} for level in ["easy_peasy", "easy", "normal", "challenging", "super_challenging"]},
    }

# Function to set the user's nickname
def set_nickname(nickname, user_state):
    if not nickname.strip():
        return user_state, "Please enter a valid nickname."
    user_state["nickname"] = nickname.strip()
    return user_state, f"Hello, {user_state['nickname']}! Select a level that you want to practice."

# Function to filter nouns by Level if the level changes
def filter_nouns_if_needed(level_with_count, user_state):
    if not user_state["nickname"]:
        return user_state, "Please set your nickname to begin."

    level = level_with_count.split(" ")[0]  # Extract level (e.g., "easy" from "easy (10 items)")
    if user_state["current_level"] != level:
        filtered_nouns = df[df["Level"] == level].copy()
        if filtered_nouns.empty:
            return user_state, f"No nouns available for the Level: {level}. Please select a different level."
        # Update the state
        user_state["remaining_nouns"] = filtered_nouns
        user_state["current_level"] = level
        user_state["score"] = 0
        user_state["trials"] = 0
        user_state["current_index"] = -1
        return user_state, f"Level {level} selected. Click 'Show the Noun' to start!"
    # If the level hasn't changed, do nothing
    return user_state, None

# Function to show the next noun
def show_next_noun(level_with_count, user_state):
    user_state, feedback = filter_nouns_if_needed(level_with_count, user_state)  # Filter nouns if needed
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})", ""
    user_state["current_index"] = random.randint(0, len(user_state["remaining_nouns"]) - 1)
    selected_noun = user_state["remaining_nouns"].iloc[user_state["current_index"]]
    return user_state, f"What's the plural form of '{selected_noun['Singular']}'?", ""

# Function to check the user's answer
def check_plural(user_plural, user_state):
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})"

    index = user_state["current_index"]
    if index == -1:
        return user_state, f"Please click 'Show the Noun' first. (Score: {user_state['score']}/{user_state['trials']})"

    noun_data = user_state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    user_state["trials"] += 1
    user_state["level_scores"][user_state["current_level"]]["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        user_state["score"] += 1
        user_state["level_scores"][user_state["current_level"]]["score"] += 1
        feedback = f"✅ Correct! '{correct_plural}' is the plural form of '{singular}' Click 'Show the Noun' to continue."
        # Remove the correctly answered noun
        user_state["remaining_nouns"] = user_state["remaining_nouns"].drop(user_state["remaining_nouns"].index[index])
    else:
        feedback = f"❌ Incorrect. The correct plural form is '{correct_plural}' for '{singular}'. This will appear again. Click 'Show the Noun' to continue."

    if user_state["remaining_nouns"].empty:
        feedback = generate_feedback(user_state["nickname"], user_state["current_level"], user_state)

    return user_state, feedback

# Function to generate constructive feedback based on performance
def generate_feedback(nickname, level, user_state):
    score = user_state["level_scores"][level]["score"]
    trials = user_state["level_scores"][level]["trials"]
    if trials == 0:
        percentage = 0
    else:
        percentage = (score / trials) * 100

    if percentage == 100:
        feedback = f"🎉 Congratulations, {nickname}! You have mastered the {level} level. Try the next level for further mastery."
    elif 80 < percentage < 100:
        feedback = f"👏 Well done, {nickname}! You are close to mastering the {level} level. A few more practices will lead you to mastery. Keep going!"
    elif 60 < percentage <= 80:
        feedback = f"👍 Good work, {nickname}! You are making steady progress in the {level} level. Keep practicing to improve further!"
    elif 40 < percentage <= 60:
        feedback = f"😊 Keep trying, {nickname}! You are halfway there in the {level} level. Practice more to strengthen your skills!"
    elif 20 < percentage <= 40:
        feedback = f"💪 Don't give up, {nickname}! You're working hard on the {level} level. Consistent effort will help you improve!"
    else:
        feedback = f"😅 Don't worry, {nickname}! The {level} level is challenging, but with persistence, you'll get better. Try again!"

    feedback += f" (Score: {score}/{trials}, {percentage:.2f}% correct)"
    return feedback

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# NounSmart: Practice Irregular Plural Nouns")

    # Nickname input
    nickname_input = gr.Textbox(label="Enter your nickname to start.")
    set_nickname_button = gr.Button("Set Nickname")
    nickname_display = gr.Textbox(label="Next Step", interactive=False)

    # Dropdown for Level selection with item counts
    level_dropdown = gr.Dropdown(
        label="Select a Level",
        choices=levels_with_counts,
        value=levels_with_counts[0],
        interactive=True
    )

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Set your nickname and select a level", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Type your answer here", placeholder="Enter the plural form")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)

    # Initialize state
    state = gr.State(initialize_user_state())

    # Button click actions
    set_nickname_button.click(fn=set_nickname, inputs=[nickname_input, state], outputs=[state, nickname_display])
    show_button.click(fn=show_next_noun, inputs=[level_dropdown, state], outputs=[state, noun_display, plural_input])
    submit_button.click(fn=check_plural, inputs=[plural_input, state], outputs=[state, feedback_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b78a913f9adeb649d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file from the provided URL
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart/refs/heads/main/Irregular_Nouns.csv"
df = pd.read_csv(csv_url)

# Count the number of items in each Level
level_counts = df["Level"].value_counts()
levels_with_counts = [
    f"{level} ({level_counts.get(level, 0)} items)"
    for level in ["easy_peasy", "easy", "normal", "challenging", "superchallenging"]
]

# Function to initialize user-specific state
def initialize_user_state():
    return {
        "nickname": None,
        "remaining_nouns": pd.DataFrame(),
        "current_level": None,
        "score": 0,
        "trials": 0,
        "current_index": -1,
        "level_scores": {level: {"score": 0, "trials": 0} for level in ["easy_peasy", "easy", "normal", "challenging", "superchallenging"]},
    }

# Function to set the user's nickname
def set_nickname(nickname, user_state):
    if not nickname.strip():
        return user_state, "Please enter a valid nickname."
    user_state["nickname"] = nickname.strip()
    return user_state, f"Hello, {user_state['nickname']}! Select a level that you want to practice."

# Function to filter nouns by Level if the level changes
def filter_nouns_if_needed(level_with_count, user_state):
    if not user_state["nickname"]:
        return user_state, "Please set your nickname to begin."

    level = level_with_count.split(" ")[0]  # Extract level (e.g., "easy" from "easy (10 items)")
    if user_state["current_level"] != level:
        filtered_nouns = df[df["Level"] == level].copy()
        if filtered_nouns.empty:
            return user_state, f"No nouns available for the Level: {level}. Please select a different level."
        # Update the state
        user_state["remaining_nouns"] = filtered_nouns
        user_state["current_level"] = level
        user_state["score"] = 0
        user_state["trials"] = 0
        user_state["current_index"] = -1
        return user_state, f"Level {level} selected. Click 'Show the Noun' to start!"
    # If the level hasn't changed, do nothing
    return user_state, None

# Function to show the next noun
def show_next_noun(level_with_count, user_state):
    user_state, feedback = filter_nouns_if_needed(level_with_count, user_state)  # Filter nouns if needed
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})", ""
    user_state["current_index"] = random.randint(0, len(user_state["remaining_nouns"]) - 1)
    selected_noun = user_state["remaining_nouns"].iloc[user_state["current_index"]]
    return user_state, f"What's the plural form of '{selected_noun['Singular']}'?", ""

# Function to check the user's answer
def check_plural(user_plural, user_state):
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})"

    index = user_state["current_index"]
    if index == -1:
        return user_state, f"Please click 'Show the Noun' first. (Score: {user_state['score']}/{user_state['trials']})"

    noun_data = user_state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    user_state["trials"] += 1
    user_state["level_scores"][user_state["current_level"]]["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        user_state["score"] += 1
        user_state["level_scores"][user_state["current_level"]]["score"] += 1
        feedback = f"✅ Correct! '{correct_plural}' is the plural form of '{singular}' Click 'Show the Noun' to continue."
        # Remove the correctly answered noun
        user_state["remaining_nouns"] = user_state["remaining_nouns"].drop(user_state["remaining_nouns"].index[index])
    else:
        feedback = f"❌ Incorrect. The correct plural form is '{correct_plural}' for '{singular}'. This will appear again. Click 'Show the Noun' to continue."

    if user_state["remaining_nouns"].empty:
        feedback = f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})"

    return user_state, feedback

# Function to display the total score
def display_total_score(user_state):
    total_score = ", ".join(
        f"{level}({user_state['level_scores'][level]['score']}/{user_state['level_scores'][level]['trials']})"
        for level in ["easy_peasy", "easy", "normal", "challenging", "superchallenging"]
    )
    return f"{user_state['nickname']}'s Total Scores: {total_score}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# NounSmart: Practice Irregular Plural Nouns")

    # Nickname input
    nickname_input = gr.Textbox(label="Enter your nickname to start.")
    set_nickname_button = gr.Button("Set Nickname")
    nickname_display = gr.Textbox(label="Next Step", interactive=False)

    # Dropdown for Level selection with item counts
    level_dropdown = gr.Dropdown(
        label="Select a Level",
        choices=levels_with_counts,
        value=levels_with_counts[0],
        interactive=True
    )

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Set your nickname and select a level", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Type your answer here", placeholder="Enter the plural form")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)

    # Total score display
    total_score_button = gr.Button("Show Total Feedback")
    total_score_display = gr.Textbox(label="Total Feedback by Levels", interactive=False)

    # Initialize state
    state = gr.State(initialize_user_state())

    # Button click actions
    set_nickname_button.click(fn=set_nickname, inputs=[nickname_input, state], outputs=[state, nickname_display])
    show_button.click(fn=show_next_noun, inputs=[level_dropdown, state], outputs=[state, noun_display, plural_input])
    submit_button.click(fn=check_plural, inputs=[plural_input, state], outputs=[state, feedback_display])
    total_score_button.click(fn=display_total_score, inputs=state, outputs=[total_score_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf43d7be873d45a27f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file from the provided URL
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart/refs/heads/main/Irregular_Nouns.csv"
df = pd.read_csv(csv_url)

# Count the number of items in each CEFR level
cefr_counts = df["CEFR Level"].value_counts()
cefr_levels_with_counts = [
    f"{level} ({cefr_counts.get(level, 0)} items)"
    for level in ["A1", "A2", "B1", "B2", "C1", "C2"]
]

# Function to initialize user-specific state
def initialize_user_state():
    return {
        "nickname": None,
        "remaining_nouns": pd.DataFrame(),
        "current_level": None,
        "score": 0,
        "trials": 0,
        "current_index": -1,
        "level_scores": {level: {"score": 0, "trials": 0} for level in ["A1", "A2", "B1", "B2", "C1", "C2"]},
    }

# Function to set the user's nickname
def set_nickname(nickname, user_state):
    if not nickname.strip():
        return user_state, "Please enter a valid nickname."
    user_state["nickname"] = nickname.strip()
    return user_state, f"Hello, {user_state['nickname']}! Select a level that you want to practice."

# Function to filter nouns by CEFR Level if the level changes
def filter_nouns_if_needed(cefr_level_with_count, user_state):
    if not user_state["nickname"]:
        return user_state, "Please set your nickname to begin."

    cefr_level = cefr_level_with_count.split(" ")[0]  # Extract level (e.g., "A1" from "A1 (10 items)")
    if user_state["current_level"] != cefr_level:
        filtered_nouns = df[df["CEFR Level"] == cefr_level].copy()
        if filtered_nouns.empty:
            return user_state, f"No nouns available for the Level: {cefr_level}. Please select a different level."
        # Update the state
        user_state["remaining_nouns"] = filtered_nouns
        user_state["current_level"] = cefr_level
        user_state["score"] = 0
        user_state["trials"] = 0
        user_state["current_index"] = -1
        return user_state, f"Level {cefr_level} selected. Click 'Show the Noun' to start!"
    # If the level hasn't changed, do nothing
    return user_state, None

# Function to show the next noun
def show_next_noun(cefr_level_with_count, user_state):
    user_state, feedback = filter_nouns_if_needed(cefr_level_with_count, user_state)  # Filter nouns if needed
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})", ""
    user_state["current_index"] = random.randint(0, len(user_state["remaining_nouns"]) - 1)
    selected_noun = user_state["remaining_nouns"].iloc[user_state["current_index"]]
    return user_state, f"What's the plural form of '{selected_noun['Singular']}'?", ""

# Function to check the user's answer
def check_plural(user_plural, user_state):
    if user_state["remaining_nouns"].empty:
        return user_state, f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})"

    index = user_state["current_index"]
    if index == -1:
        return user_state, f"Please click 'Show the Noun' first. (Score: {user_state['score']}/{user_state['trials']})"

    noun_data = user_state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    user_state["trials"] += 1
    user_state["level_scores"][user_state["current_level"]]["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        user_state["score"] += 1
        user_state["level_scores"][user_state["current_level"]]["score"] += 1
        feedback = f"✅ Correct! '{correct_plural}' is the plural form of '{singular}' Click 'Show the Noun' to continue."
        # Remove the correctly answered noun
        user_state["remaining_nouns"] = user_state["remaining_nouns"].drop(user_state["remaining_nouns"].index[index])
    else:
        feedback = f"❌ Incorrect. The correct plural form is '{correct_plural}' for '{singular}'. This will appear again. Click 'Show the Noun' to continue."

    if user_state["remaining_nouns"].empty:
        feedback = f"🎉 Great job, {user_state['nickname']}! All nouns have been answered correctly. (Score: {user_state['score']}/{user_state['trials']})"

    return user_state, feedback

# Function to display the total score
def display_total_score(user_state):
    total_score = ", ".join(
        f"{level}({user_state['level_scores'][level]['score']}/{user_state['level_scores'][level]['trials']})"
        for level in ["A1", "A2", "B1", "B2", "C1", "C2"]
    )
    return f"{user_state['nickname']}'s Total Scores: {total_score}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# NounSmart: Practice Irregular Plural Nouns")

    # Nickname input
    nickname_input = gr.Textbox(label="Enter your nickname to start.")
    set_nickname_button = gr.Button("Set Nickname")
    nickname_display = gr.Textbox(label="Next Step", interactive=False)

    # Dropdown for CEFR Level selection with item counts
    cefr_dropdown = gr.Dropdown(
        label="Select a Level",
        choices=cefr_levels_with_counts,
        value=cefr_levels_with_counts[0],
        interactive=True
    )

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Set your nickname and select a level", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Type your answer here", placeholder="Enter the plural form")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)

    # Total score display
    total_score_button = gr.Button("Show Total Scores")
    total_score_display = gr.Textbox(label="Total Score by Levels", interactive=False)

    # Initialize state
    state = gr.State(initialize_user_state())

    # Button click actions
    set_nickname_button.click(fn=set_nickname, inputs=[nickname_input, state], outputs=[state, nickname_display])
    show_button.click(fn=show_next_noun, inputs=[cefr_dropdown, state], outputs=[state, noun_display, plural_input])
    submit_button.click(fn=check_plural, inputs=[plural_input, state], outputs=[state, feedback_display])
    total_score_button.click(fn=display_total_score, inputs=state, outputs=total_score_display)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2f67638f48d6f7831.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
